In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
# 数值特征
import pandas as pd
import numpy as np
import json
from tqdm import tqdm as tqdm
from sklearn.model_selection import KFold
from gensim.models import word2vec
from gensim.models import Word2Vec

In [ ]:
data = pd.read_pickle('data/step1.pkl')
test_data = pd.read_pickle('data/step1_test.pkl')
print(data.shape,test_data.shape)

In [ ]:
bert_dis = pd.read_pickle('data/bert_dis.pkl')
bert_dis_test = pd.read_pickle('data/bert_dis.pkl')
print(bert_dis.shape,bert_dis_test.shape)

In [ ]:
data['bert_cos'] = bert_dis['bert_cos']
data['bert_point'] = bert_dis['bert_point']
data['bert_mse'] = bert_dis['bert_mse']

In [ ]:
test_data['bert_cos'] = bert_dis_test['bert_cos']
test_data['bert_point'] = bert_dis_test['bert_point']
test_data['bert_mse'] = bert_dis_test['bert_mse']

In [ ]:
test_data['label'] = -1
data_all = data.append(test_data).reset_index(drop=True)

In [ ]:
numerical_clolumns = ['bert_cos','bert_point','bert_mse']

In [ ]:
# rank 特征
to_rank = numerical_clolumns
for f in to_rank:
    data_group = data_all.groupby('kb_id')[f].rank(ascending=False)
    data_all['kbid_rank_'+f] = data_group

In [ ]:
# mean and var
to_rank = numerical_clolumns
for f in tqdm(to_rank):
    data_group = data_all.groupby('kb_id')[f].agg({'kb_'+f+'_var':'var','kb_'+f+'_mean':'mean','kb_'+f+'_max':'max',
                                                   'kb_'+f+'_min':'min'})
    data_all = data_all.merge(data_group,on='kb_id',how='left')

In [ ]:
# mean and var
to_rank = numerical_clolumns
for f in tqdm(to_rank):
    data_group = data_all.groupby(['kb_id','train_mention'])[f].agg({'kb_m_'+f+'_var':'var','kb_m_'+f+'_mean':'mean',
                                                                 'kb_m_'+f+'_max':'max',
                                                   'kb_m_'+f+'_min':'min'})
    data_all = data_all.merge(data_group,on=['kb_id','train_mention'],how='left')

In [ ]:
# rank 特征
to_rank = numerical_clolumns
for f in to_rank:
    data_group = data_all.groupby(['kb_id','train_mention'])[f].rank(ascending=False)
    data_all['kb_m_rank_'+f] = data_group
   

In [ ]:
for f in to_rank:
    data_group = data_all.groupby('m_id')[f].rank(ascending=False)
    data_all['m_rank_'+f] = data_group

In [ ]:
for f in tqdm(to_rank):
    data_group = data_all.groupby('m_id')[f].agg({'mid_'+f+'_var':'var','mid_'+f+'_mean':'mean',
                                                  'mid_'+f+'_max':'max','mid_'+f+'_min':'min'})
    data_all = data_all.merge(data_group,on='m_id',how='left')

In [ ]:
for item in data_all.columns:
    if data_all.dtypes[item]=='float64':
        print(item)
        data_all[item] = data_all[item].astype('float32')
    if data_all.dtypes[item]=='int64':
        data_all[item] = data_all[item].astype('int32')
    if data_all.dtypes[item]=='bool':
        data_all[item] = data_all[item].astype('int8')

In [ ]:
data = data_all.iloc[0:data.shape[0],:]
test_data = data_all.iloc[data.shape[0]:,:]
data.reset_index(drop=True,inplace=True)
test_data.reset_index(drop=True,inplace=True)

In [ ]:
data.to_pickle('data/step7.pkl')
test_data.to_pickle('data/step7_test.pkl')